#### pyMBE : The python-based Molecule Builder for ESPResSo.
#### Copyright (C) 2023 - 2024 pyMBE-dev team

This file is part of pyMBE.

pyMBE is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

pyMBE is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.

 # pyMBE Tutorial Solution: Alternating copolymer.





This jupyter notebook presentes the solution to the proposed excise of creating an alternating PAA-PVAm copolymer (Polyacrylic acid - Polyvinylamine), whose structure is depicted in the figure below.

<img src="../figs/PAA_PVAm.png" width=250 height=250 />

Let us get started by importing pyMBE library and other important libraries for this exercise, by creating an instance of pyMBE and by creating the simulation box.

In [ ]:
# Import pyMBE and  ESPResSo
import pyMBE
import espressomd
from espressomd import interactions

# Only necesary to produce the pictures used in this tutorial
from lib.handy_functions import do_snapshot_espresso_system
from PIL import Image


pmb = pyMBE.pymbe_library(SEED=42)

pmb.set_reduced_units(unit_length = 1.5*pmb.units.nm,  
                      unit_charge = 5*pmb.units.e)

Box_L = 7.5*pmb.units.nm

espresso_system = espressomd.System(box_l = [Box_L.to('reduced_length').magnitude]*3)

print('The side of the simulation box is ', Box_L, '=' ,Box_L.to('reduced_length'))

### Tasks to do:

1. Define each different bead of the PAA and PVAm.
2. Define the different residues for the PAA and PVAm.
3. Define the alternating block copolymer molecule. 
4. Create the alternating block copolymer into the espresso system.
5. Visulalize your creation.
6. Delete the molecule and check that your df is empty.


#### 1. Define each different bead of the PAA and PVAm.

In [ ]:
PAA_backbone_bead = 'BB-PAA'
PAA_carboxyl_bead = 'COOH-PAA'

pmb.define_particle(name = PAA_backbone_bead, 
                    q = 0,
                    sigma = 0.4*pmb.units.nm,
                    epsilon = 1*pmb.units('reduced_energy'))

pmb.define_particle(name = PAA_carboxyl_bead,
                    q = 0,
                    sigma = 0.5*pmb.units.nm,
                    epsilon = 1*pmb.units('reduced_energy'))

PVam_backbone_bead = 'BB-PVam'
PVam_amine_bead = 'NH3-PVam'

pmb.define_particle(name = PVam_backbone_bead,
                    q = 0,
                    sigma = 0.4*pmb.units.nm, 
                    epsilon = 1*pmb.units('reduced_energy'))

pmb.define_particle(name = PVam_amine_bead,
                    q = 0, 
                    sigma = 0.3*pmb.units.nm,
                    epsilon = 1*pmb.units('reduced_energy'))

#### 2. Define the different residues and the bonds for the PAA and PVAm.

In [ ]:
PAA_residue = 'PAA_mon'
pmb.define_residue(name = PAA_residue, 
                   central_bead =  PAA_backbone_bead ,
                   side_chains = [PAA_carboxyl_bead])

PVam_residue = 'PVam_mon'
pmb.define_residue( name = PVam_residue,
                    central_bead = PVam_backbone_bead,
                    side_chains = [PVam_amine_bead])

bond_type = 'harmonic'
generic_bond_lenght=0.4 * pmb.units.nm
generic_harmonic_constant = 400 * pmb.units('reduced_energy / reduced_length**2')

harmonic_bond = {'r_0'    : generic_bond_lenght,
                 'k'      : generic_harmonic_constant}

pmb.define_bond(bond_type = bond_type,
                bond_parameters = harmonic_bond,
                particle_pairs = [[PAA_backbone_bead, PAA_carboxyl_bead],
                                  [PVam_backbone_bead, PVam_amine_bead],
                                  [PAA_backbone_bead, PVam_backbone_bead]])

pmb.add_bonds_to_espresso(espresso_system = espresso_system)

#### 3. Define the alternating block copolymer molecule. 

In [ ]:
alternating_polymer = 'alternating'
N_dimers = 4

pmb.define_molecule(name = alternating_polymer,
                    residue_list = [PAA_residue, PVam_residue]*N_dimers)

#### 4. Create the alternating block copolymer into the espresso system.

In [ ]:
N_polymers = 1

pmb.create_pmb_object(name = alternating_polymer,
                      number_of_objects = N_polymers,
                      espresso_system = espresso_system,
                      position = [[Box_L.to('reduced_length').magnitude/2]*3]) 

#### 5. Visualize your creation.

In [ ]:
picture_name = 'alternating_system.png'
do_snapshot_espresso_system(espresso_system = espresso_system, 
                               filename = picture_name)
img = Image.open(picture_name)
img.show()

#### 6. Delete the molecule and check that your df is empty.

In [ ]:
pmb.destroy_pmb_object_in_system(name = alternating_polymer, 
                                 espresso_system = espresso_system)
pmb.filter_df(pmb_type = 'particle')